# Experiment 1 Baseline

Checking whether an array is a heap using an RNN or LSTM

In [3]:
%cd ..
%pwd
import wandb
from experiments.heaps import *
from experiments.utils import *
from baselines.baseline_rnn import *
from baselines.baseline_lstm import *
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

/home/tekne/Oxford/ATML/Project


In [4]:
def get_data(config):
    data = make_heap_test_rnn_datapoints(
        n = config["n_train"] + config["n_test"],
        p_heap = config["p_heap"],
        min_len = config["min_len"],
        max_len = config["max_len"],
    )

    training_data = DataLoader(data[:config["n_train"]], batch_size=config["batch_size"])
    testing_data = DataLoader(data[config["n_train"]:], batch_size=config["batch_size"])

    return training_data, testing_data

def make_lstm(config):
    return BaselineLSTM(
        input_size=1,
        hidden_size=config["hidden_size"]
    ).cuda()

def make_rnn(config):
    return BaselineRNN(
        input_size=1,
        hidden_size=config["hidden_size"]
    ).cuda()

def make_model(config):
    model = config.get("model", "rnn")
    if model == "rnn":
        make_lstm(config)
    elif model == "lstm":
        make_rnn(config)
    else:
        raise ValueError(f"config[\"model\"] == {model}")

In [5]:
def run_model(config, logger=wandb):
    NUM_CLASSES = 2

    model = GraphLevelGGNN(
        annotation_size=1,
        num_layers=config["num_layers"],
        gate_nn=nn.Linear(2 * 1 + config["hidden_size"], 1),
        hidden_size=config["hidden_size"],
        final_layer=nn.Linear(2 * 1 + config["hidden_size"], NUM_CLASSES),
    ).cuda()

    if logger is not None:
        logger.watch(model)
        logger.run.summary["model_parameters"] = model.count_parameters()

    opt = torch.optim.Adam(model.parameters(), lr=config["lr"])
    loss = torch.nn.CrossEntropyLoss()

    training_data, testing_data = get_data(config)

    results = train(
        model=model,
        opt=opt,
        training_data=training_data,
        testing_data=testing_data,
        criterion=lambda out, y: loss(out, y),
        checker=lambda out, y: ((torch.argmax(out, dim=-1) == y).sum(), y.shape[0]),
        epochs=config["epochs"],
        logger=logger
    )

    plt.plot(results["train_loss"], label="Training loss")
    plt.plot(results["test_loss"], label="Testing loss")
    plt.legend()
    plt.show()

    plt.plot(results["train_accuracy"], label="Training accuracy")
    plt.plot(results["test_accuracy"], label="Testing accuracy")
    plt.legend()
    plt.show()
